<a href="https://colab.research.google.com/github/sanquaorg/Deep-Learning---/blob/main/Rain_prediction_using_Feed_Forward_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from tensorflow import keras #For building neural networks
from keras.models import Sequential #For creating a linear stack of layers for our Neural Networks
from keras import Input #For instantiationg a keras tensor
from keras.layers import Dense #For creating regularly densely-connected NN layer
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import plotly
import plotly.express as px

In [4]:
pd.options.display.max_columns=50
df=pd.read_csv('https://raw.githubusercontent.com/sanquaorg/Deep-Learning---/main/weatherAUS.csv',encoding='utf-8')
df=df[pd.isnull(df['RainTomorrow'])==False]
df=df.fillna(df.mean())
df['RainTodayFlag']=df['RainToday'].apply(lambda x:1 if x=='Yes' else 0)
df['RainTomorrowFlag']=df['RainTomorrow'].apply(lambda x:1 if x=='Yes' else 0)
df

<ipython-input-4-4b4a6bccc25d>:4: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,RainTodayFlag,RainTomorrowFlag
0,2008-12-01,Albury,13.4,22.9,0.6,5.469824,7.624853,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.000000,4.503167,16.9,21.8,No,No,0,0
1,2008-12-02,Albury,7.4,25.1,0.0,5.469824,7.624853,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,4.437189,4.503167,17.2,24.3,No,No,0,0
2,2008-12-03,Albury,12.9,25.7,0.0,5.469824,7.624853,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,4.437189,2.000000,21.0,23.2,No,No,0,0
3,2008-12-04,Albury,9.2,28.0,0.0,5.469824,7.624853,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,4.437189,4.503167,18.1,26.5,No,No,0,0
4,2008-12-05,Albury,17.5,32.3,1.0,5.469824,7.624853,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.000000,8.000000,17.8,29.7,No,No,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145454,2017-06-20,Uluru,3.5,21.8,0.0,5.469824,7.624853,E,31.0,ESE,E,15.0,13.0,59.0,27.0,1024.7,1021.2,4.437189,4.503167,9.4,20.9,No,No,0,0
145455,2017-06-21,Uluru,2.8,23.4,0.0,5.469824,7.624853,E,31.0,SE,ENE,13.0,11.0,51.0,24.0,1024.6,1020.3,4.437189,4.503167,10.1,22.4,No,No,0,0
145456,2017-06-22,Uluru,3.6,25.3,0.0,5.469824,7.624853,NNW,22.0,SE,N,13.0,9.0,56.0,21.0,1023.5,1019.1,4.437189,4.503167,10.9,24.5,No,No,0,0
145457,2017-06-23,Uluru,5.4,26.9,0.0,5.469824,7.624853,N,37.0,SE,WNW,9.0,9.0,53.0,24.0,1021.0,1016.8,4.437189,4.503167,12.5,26.1,No,No,0,0


->Using Humidity3pm column as a single input variable for predicting whether it will rain tomorrow or not.

->ONE I/P NODE - 2 HIDDEN NODE - 1 O/P NODE


In [15]:
x = df[['Humidity3pm']]
y = df['RainTomorrowFlag'].values

X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

model=Sequential(name="Model-with-One-Input") #Model
model.add(Input(1,name='Input-Layer')) #Input alyer - need to specify the shape of inputs
model.add(Dense(2,activation='softplus',name='Hidden-Layer'))# Hidden layer, softplus= log(exp(x) +1)
model.add(Dense(1,activation='sigmoid',name='Output-Layer'))#Output layer, sigmoid=1/1+exp(-x)

model.compile(optimizer='adam', #adam is algo used in backpropogation
              loss='binary_crossentropy', #Loss function to be optimized.
              metrics=['Accuracy','Precision','Recall'],
              loss_weights = None, #Scalar coeff to weight the loss contributions of different model outputs
              run_eagerly=None,
              steps_per_execution=None)



In [16]:
model.fit(X_train,y_train,batch_size=10, #Number of samples per gradient update
          epochs=3,verbose='auto',callbacks=None,validation_split=0.2,
          shuffle=True,class_weight={0:0.3,1:0.7},
          sample_weight=None,
          initial_epoch=0,
          steps_per_epoch=None,
          validation_steps=None,
          validation_batch_size=None,
          validation_freq=3,
          max_queue_size=10,
          workers=1,
          use_multiprocessing=False)

Epoch 1/3
9101/9101 [==============================] - 21s 2ms/step - loss: 1.2381 - Accuracy: 0.6716 - precision: 0.3728 - recall: 0.6795
Epoch 2/3
9101/9101 [==============================] - 20s 2ms/step - loss: 0.2100 - Accuracy: 0.7759 - precision: 0.5005 - recall: 0.5977
Epoch 3/3
9101/9101 [==============================] - 24s 3ms/step - loss: 0.2093 - Accuracy: 0.7765 - precision: 0.5017 - recall: 0.5956 - val_loss: 0.4670 - val_Accuracy: 0.7974 - val_precision: 0.5454 - val_recall: 0.5663


In [17]:
pred_labels_tr=(model.predict(X_train)>0.5).astype(int)
pred_labels_te=(model.predict(X_test)>0.5).astype(int)


889/889 [==============================] - 2s 3ms/step


In [19]:
print("")
print('Model Summary:')
model.summary()
print("")
print("Weights and biases")
for layer in model.layers:
  print("Layer:", layer.name)
  print("Kernels(Weights):",layer.get_weights()[0])
  print("Biases",layer.get_weights()[1])

print("")
print("Evaluation on training data")
print(classification_report(y_train,pred_labels_tr))
print("")

print("Evaluation on testing data")
print(classification_report(y_test,pred_labels_te))
print("")


Model Summary:
Model: "Model-with-One-Input"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden-Layer (Dense)        (None, 2)                 4         
                                                                 
 Output-Layer (Dense)        (None, 1)                 3         
                                                                 
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________

Weights and biases
Layer: Hidden-Layer
Kernels(Weights): [[ 0.16177446 -0.04949247]]
Biases [-2.3006148  -0.50214195]
Layer: Output-Layer
Kernels(Weights): [[0.3690399]
 [0.9015088]]
Biases [-3.1148057]

Evaluation on training data
              precision    recall  f1-score   support

           0       0.87      0.86      0.87     88249
           1       0.54      0.55      0.55     25505

    accuracy                       

[[0]
 [0]
 [0]
 ...
 [0]
 [1]
 [1]]
